### Save results to `.pkl` and `.npz`

In [1]:
import config,csv,utils,json,pickle,os
import numpy as np
from tqdm.notebook import tqdm
print ("Done.")

Done.


### Parse `./video_path.csv`

In [2]:
csv_path = './video_path.csv'
name_and_time = dict()
with open(csv_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    cnt = 0
    for row in csv_reader:
        if cnt > 0: # skip the first line (header)
            name_and_time[row[0]] = (float(row[1]), float(row[2]))
        cnt += 1
# Get video IDs
video_ids = []
for vid_name,(s_t,e_t) in name_and_time.items():
    print ("Name:[%s] [%.2f]s~[%.2f]s"%(vid_name,s_t,e_t))
    video_ids.append(vid_name)
n_vid = len(video_ids)
print ("video_ids:%s"%(video_ids))

Name:[MVI_0005] [0.00]s~[0.00]s
Name:[1X5A1788] [0.00]s~[0.00]s
Name:[1X5A1789] [0.00]s~[0.00]s
Name:[1X5A1790] [0.00]s~[0.00]s
Name:[MVI_0003] [0.00]s~[0.00]s
Name:[MVI_0004] [0.00]s~[0.00]s
video_ids:['MVI_0005', '1X5A1788', '1X5A1789', '1X5A1790', 'MVI_0003', 'MVI_0004']


### Results will be saved to
- `./skeletons_pkl` and `./skeletons_npz`

In [3]:
os.makedirs(config.skeleton_pkl_dir, exist_ok=True)
os.makedirs(config.skeleton_npz_dir, exist_ok=True)
print ("[./%s] created."%(config.skeleton_pkl_dir))
print ("[./%s] created."%(config.skeleton_npz_dir))

[./skeletons_pkl] created.
[./skeletons_npz] created.


### Loop

In [4]:
for v_idx,video_id in enumerate(video_ids):
    result_dir = os.path.join(config.mocap_dir,video_id)
    bbox_dir = os.path.join(result_dir, 'bbox')
    mocap_dir = os.path.join(result_dir, 'mocap')
    print ("[%d] [%s]"%(v_idx,video_id))
    
    # Create directories (if not exist)
    os.makedirs(bbox_dir,exist_ok=True)
    os.makedirs(mocap_dir,exist_ok=True)
    pkl_dir = os.path.join(config.skeleton_pkl_dir,video_id)
    npz_dir = os.path.join(config.skeleton_npz_dir,video_id)
    os.makedirs(pkl_dir, exist_ok=True)
    os.makedirs(npz_dir, exist_ok=True)
    
    print (" [%s] created."%(bbox_dir))
    print (" [%s] created."%(mocap_dir))
    print (" [%s] created."%(pkl_dir))
    print (" [%s] created."%(npz_dir))
    
    for mocap_fp in tqdm(sorted(os.listdir(mocap_dir))): # for all frames
        scene_name = mocap_fp.split('/')[-1][:14]
        
        bbox_fp = os.path.join(bbox_dir, scene_name+'_bbox.json')
        mocap_fp = os.path.join(mocap_dir, scene_name+'_prediction_result.pkl')
        
        ### load info
        bbox = json.load(open(bbox_fp, 'rb'))
        mocap = pickle.load(open(mocap_fp, 'rb'))
        
        ### get bbox size
        bbox_np = np.array(bbox['body_bbox_list'])
        # bbox_size = [bb[2]*bb[3] for bb in bbox_np]
        bbox_size = [bb[0] for bb in bbox_np]

        ### only choose the biggest bbox
        max_idx = np.argmax(bbox_size)
        curr_body_bbox = bbox_np[max_idx]
        curr_mocap = mocap['pred_output_list'][max_idx]

        ### extract body and hand pose
        curr_body_pose = curr_mocap['pred_joints_img']
        curr_open_pose = curr_body_pose[:25]
        curr_aux_pose = curr_body_pose[25:]

        curr_right_hand = curr_mocap['right_hand_joints_img_coord']
        curr_left_hand = curr_mocap['left_hand_joints_img_coord']
        
        # extract hand bbox
        if None not in mocap['hand_bbox_list']:
            curr_left_hand_bbox = np.asarray(mocap['hand_bbox_list'][max_idx]['left_hand'])
            curr_right_hand_bbox = np.asarray(mocap['hand_bbox_list'][max_idx]['right_hand'])
            if curr_left_hand_bbox.all() == None:
                curr_left_hand_bbox = np.array([0, 0, 0, 0])
            if curr_right_hand_bbox.all() == None:
                curr_right_hand_bbox = np.array([0, 0, 0, 0])
        else:
            curr_left_hand_bbox = np.array([0, 0, 0, 0])
            curr_right_hand_bbox = np.array([0, 0, 0, 0])
            
        ### extract simplified pose
        curr_simple_pose = curr_open_pose[:15] # 15 by 3

        ### process for .npz file.
        left_hand_info, left_hand_array = utils.get_hand_info(curr_left_hand)
        right_hand_info, right_hand_array = utils.get_hand_info(curr_right_hand)

        left_heel_info, right_heel_info, heel_array = utils.get_heel_info(curr_open_pose)

        head_info, head_array = utils.get_head_info(curr_open_pose)

        ### prepare dictionary
        pkl_saver = dict()
        pkl_saver['open_pose'] = curr_open_pose
        pkl_saver['aux_pose'] = curr_aux_pose
        pkl_saver['simple_pose'] = curr_simple_pose
        pkl_saver['left_hand'] = curr_left_hand
        pkl_saver['right_hand'] = curr_right_hand
        pkl_saver['hand_info'] = {'left':left_hand_info, 'right':right_hand_info}
        pkl_saver['heel_info'] = {'left':left_heel_info, 'right':right_heel_info}
        pkl_saver['head_info'] = head_info
        pkl_saver['body_bbox'] = curr_body_bbox
        pkl_saver['left_hand_bbox'] = curr_left_hand_bbox
        pkl_saver['right_hand_bbox'] = curr_right_hand_bbox

        ### prepare the array
        npz_saver = [curr_simple_pose.flatten(), head_array, left_hand_array, right_hand_array, heel_array, 
                     curr_body_bbox, curr_left_hand_bbox, curr_right_hand_bbox]
        npz_saver = np.concatenate(npz_saver)
        # 0  ~ 45 : pose that was 15 by 3
        # 45 ~ 51 : head pos(3), vec(3)
        # 51 ~ 63 : left hand thumb vector(3), index vector(3), palm vector(3), wrist pos(3)
        # 63 ~ 75 : right hand thumb vector(3), index vector(3), palm vector(3), wrist pos(3)
        # 75 ~ 87 : left heel pos (3), left foot vec (3), right heel pos (3), right foot vec (3)
        # 87 ~ 91 : body bbox (4) left hand bbox (4) right hand bbox(4)

        ### save the data
        pkl_fp = os.path.join(pkl_dir, scene_name+'.pkl')
        npz_fp = os.path.join(npz_dir, scene_name+'.npz')
        pickle.dump(pkl_saver, open(pkl_fp, 'wb')) # save pickle 
        np.savez(npz_fp, npz_saver) # save npz 
        
print ("Done.")

[0] [MVI_0005]
 [mocaps/MVI_0005/bbox] created.
 [mocaps/MVI_0005/mocap] created.
 [skeletons_pkl/MVI_0005] created.
 [skeletons_npz/MVI_0005] created.



[1] [1X5A1788]
 [mocaps/1X5A1788/bbox] created.
 [mocaps/1X5A1788/mocap] created.
 [skeletons_pkl/1X5A1788] created.
 [skeletons_npz/1X5A1788] created.



[2] [1X5A1789]
 [mocaps/1X5A1789/bbox] created.
 [mocaps/1X5A1789/mocap] created.
 [skeletons_pkl/1X5A1789] created.
 [skeletons_npz/1X5A1789] created.



[3] [1X5A1790]
 [mocaps/1X5A1790/bbox] created.
 [mocaps/1X5A1790/mocap] created.
 [skeletons_pkl/1X5A1790] created.
 [skeletons_npz/1X5A1790] created.



[4] [MVI_0003]
 [mocaps/MVI_0003/bbox] created.
 [mocaps/MVI_0003/mocap] created.
 [skeletons_pkl/MVI_0003] created.
 [skeletons_npz/MVI_0003] created.



[5] [MVI_0004]
 [mocaps/MVI_0004/bbox] created.
 [mocaps/MVI_0004/mocap] created.
 [skeletons_pkl/MVI_0004] created.
 [skeletons_npz/MVI_0004] created.



Done.
